## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-05-08-41-58 +0000,bbc,Poland scrambles jets as Ukraine says 5 killed...,https://www.bbc.com/news/articles/czjvlgzmp4wo...
1,2025-10-05-08-36-25 +0000,bbc,Home secretary wants police to get more powers...,https://www.bbc.com/news/articles/c24rmdngrrjo...
2,2025-10-05-08-33-28 +0000,nyt,Typhoon Matmo Makes Landfall in Southern China,https://www.nytimes.com/2025/10/05/world/asia/...
3,2025-10-05-08-25-01 +0000,bbc,Wafcon expands to 16 teams for 2026 finals,https://www.bbc.com/sport/football/articles/c0...
4,2025-10-05-08-00-00 +0000,cbc,Conservatives won't support Liberal border bil...,https://www.cbc.ca/news/politics/conservatives...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2355/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
51,trump,27
54,gaza,18
56,israel,11
440,plan,10
292,deal,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
31,2025-10-05-02-19-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...,105
74,2025-10-04-20-36-56 +0000,nypost,Israel agrees to ‘initial withdrawal line’ in ...,https://nypost.com/2025/10/04/world-news/israe...,97
131,2025-10-04-14-18-43 +0000,nypost,Hamas’ response to Trump’s Gaza peace plan win...,https://nypost.com/2025/10/04/world-news/hamas...,91
136,2025-10-04-13-57-38 +0000,bbc,Shock in Gaza as Trump appears to welcome Hama...,https://www.bbc.com/news/articles/c15k199j1x3o...,78
101,2025-10-04-17-42-17 +0000,nypost,Trump reveals Netanyahu phone call that moved ...,https://nypost.com/2025/10/04/world-news/trump...,75


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
31,105,2025-10-05-02-19-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...
74,70,2025-10-04-20-36-56 +0000,nypost,Israel agrees to ‘initial withdrawal line’ in ...,https://nypost.com/2025/10/04/world-news/israe...
111,41,2025-10-04-16-28-25 +0000,nypost,London police arrest dozens of protesters agai...,https://nypost.com/2025/10/04/world-news/londo...
148,39,2025-10-04-12-37-30 +0000,cbc,Russian drone strike hits passenger train in U...,https://www.cbc.ca/news/world/russia-drone-ukr...
157,39,2025-10-04-12-00-00 +0000,wsj,Many government leaders have had the chance to...,https://www.wsj.com/economy/trade/south-korea-...
24,35,2025-10-05-04-01-05 +0000,nyt,Syria Holds First Parliamentary Elections Sinc...,https://www.nytimes.com/2025/10/05/world/middl...
52,30,2025-10-04-23-05-09 +0000,nypost,Youngkin calls for embattled Dem AG hopeful Ja...,https://nypost.com/2025/10/04/us-news/virginia...
51,30,2025-10-04-23-14-00 +0000,wsj,Defense Secretary Pete Hegseth’s emphasis on p...,https://www.wsj.com/politics/national-security...
77,28,2025-10-04-20-11-06 +0000,nyt,Andrej Babis Likely to Return as Czech Prime M...,https://www.nytimes.com/2025/10/04/world/europ...
131,28,2025-10-04-14-18-43 +0000,nypost,Hamas’ response to Trump’s Gaza peace plan win...,https://nypost.com/2025/10/04/world-news/hamas...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
